In [4]:
! pip install groq
from groq import Groq
import json


In [5]:
client = Groq(api_key="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")


In [6]:
# Function schema definition for structured extraction
functions = [
    {
        "name": "extract_user_info",
        "description": "Extract personal details from chat conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Full name of the user"},
                "email": {"type": "string", "description": "Email address"},
                "phone": {"type": "string", "description": "Phone number"},
                "location": {"type": "string", "description": "Location or city"},
                "age": {"type": "integer", "description": "Age in years"}
            },
            "required": ["name", "email", "phone", "location", "age"]
        }
    }
]


In [7]:
def extract_info_from_chat(chat_text, model="llama-3.1-8b-instant"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an information extraction assistant."},
            {"role": "user", "content": chat_text}
        ],
        functions=functions,
        function_call={"name": "extract_user_info"}
    )

    # Get structured function call output
    msg = response.choices[0].message
    if msg.function_call and msg.function_call.arguments:
      function_args = msg.function_call.arguments
    else:
      function_args = "{}"

    data = json.loads(function_args)


    return data


In [8]:
sample_chats = [
    "Hello, my name is John Doe. I live in New York. You can reach me at john@example.com or call 555-1234. I'm 29 years old.",
    "Hi, I'm Alice. Email: alice99@mail.com, Phone: 9876543210. I'm 34 years old and currently living in London.",
    "This is Raj from Bangalore, 41 years old. My contact is rajkumar@mail.in and phone is 9090909090."
]

for i, chat in enumerate(sample_chats, 1):
    extracted = extract_info_from_chat(chat)
    print(f"\n--- Chat {i} ---")
    print("Input:", chat)
    print("Extracted JSON:", extracted)



--- Chat 1 ---
Input: Hello, my name is John Doe. I live in New York. You can reach me at john@example.com or call 555-1234. I'm 29 years old.
Extracted JSON: {'age': 29, 'email': 'john@example.com', 'location': 'New York', 'name': 'John Doe', 'phone': '555-1234'}

--- Chat 2 ---
Input: Hi, I'm Alice. Email: alice99@mail.com, Phone: 9876543210. I'm 34 years old and currently living in London.
Extracted JSON: {'age': 34, 'email': 'alice99@mail.com', 'location': 'London', 'name': 'Alice', 'phone': '9876543210'}

--- Chat 3 ---
Input: This is Raj from Bangalore, 41 years old. My contact is rajkumar@mail.in and phone is 9090909090.
Extracted JSON: {'age': 41, 'email': 'rajkumar@mail.in', 'location': 'Bangalore', 'name': 'Raj', 'phone': '9090909090'}


In [9]:
from jsonschema import validate, ValidationError

# Define schema for validation (same as function parameters)
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

for i, chat in enumerate(sample_chats, 1):
    extracted = extract_info_from_chat(chat)
    print(f"\n--- Validating Chat {i} ---")
    try:
        validate(instance=extracted, schema=schema)
        print("✅ Valid output:", extracted)
    except ValidationError as e:
        print("❌ Validation failed:", e)



--- Validating Chat 1 ---
✅ Valid output: {'age': 29, 'email': 'john@example.com', 'location': 'New York', 'name': 'John Doe', 'phone': '555-1234'}

--- Validating Chat 2 ---
✅ Valid output: {'age': 34, 'email': 'alice99@mail.com', 'location': 'London', 'name': 'Alice', 'phone': '9876543210'}

--- Validating Chat 3 ---
✅ Valid output: {'age': 41, 'email': 'rajkumar@mail.in', 'location': 'Bangalore', 'name': 'Rajkumar', 'phone': '9090909090'}
